In [1]:
import pandas as pd

In [2]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)

# Import csv file
data = pd.read_csv('final_data.csv', encoding='unicode_escape', index_col=0)
data.head(3)

,beer_name,user,aroma,appearance,flavor,mouthfeel,overall,rating,date
1,San Miguel,ResinousMaestro55(268),5,3,5,3,9,2.5,15-Jan-21
2,San Miguel,"nimbleprop(11,986)",5,3,5,2,10,2.5,09-Feb-20
3,San Miguel,"troopie(4,238)",2,3,3,1,4,1.3,13-Jun-19


In [8]:
# Delete duplicated "beer name"
beer_data = data.copy()
beer_data = beer_data[['beer_name']]

beer_data.drop_duplicates(keep='first', inplace=True)
cols = ['aroma', 'appearance','flavor','mouthfeel','overall']

# Create 5 categories for column name
for col in cols:
    beer_data[col] = ''
beer_data.head(5)

,beer_name,aroma,appearance,flavor,mouthfeel,overall
1,San Miguel,,,,,
192,Newcastle Brown Ale (Non-US Version),,,,,
4125,Rickard's Red,,,,,
4621,Paradox Beer Candid Kaiser - American Style Pa...,,,,,
4628,Maclay's Traditional Pale Ale,,,,,


In [10]:
# Calculate average score for each columns
for i in range(len(beer_data)):
   
    # collect beer name
    beer = beer_data['beer_name'].iloc[i]

    # the total number of reviewers
    length = len(data[data['beer_name']==beer])
    # calculate the each columns 
    for col in cols:
        # total score for each columns 
        col_sum = data[data['beer_name']==beer][col].sum()
        # average score for each columns
        beer_data[col].iloc[i] = col_sum/length

beer_data.to_csv('beer_score.csv', encoding='utf-8')
beer_data.head(5)

,beer_name,aroma,appearance,flavor,mouthfeel,overall
1,San Miguel,3.63351,2.5445,3.82199,2.2199,7.60733
192,Newcastle Brown Ale (Non-US Version),5.4152,3.41114,5.92525,3.01831,12.107
4125,Rickard's Red,4.77218,3.25202,5.12097,2.6875,10.2823
4621,Paradox Beer Candid Kaiser - American Style Pa...,8,3.85714,7.14286,4.14286,15
4628,Maclay's Traditional Pale Ale,3.63303,2.33945,4.06422,2.20183,8
